<a href="https://colab.research.google.com/github/andreluismiller/ETL_python/blob/main/Projeto_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importação biblioteca pandas

import pandas as pd

In [ ]:
#Instalação e importação pandera

!pip install pandera
import pandera as pa

In [3]:
#Importando Numpy para evitar erro FutureWarning no schema

import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
print('x' in np.arange(5))   #returns False, without Warning

False


In [4]:
#Carregar arquivo já alterando tipo de dado da coluna ocorrencia_dia
valores_ausentes = ["**", "###!", "####", "*****", "****", "NULL"]
df = pd.read_csv("/content/drive/MyDrive/ETL Python/ocorrencia_2010_2020 (1).csv", encoding="utf-8", sep=";", 
                 parse_dates=["ocorrencia_dia"], dayfirst=True, na_values=valores_ausentes)

In [5]:
#Apagar colunas do dataframe que não interessa

df.drop(columns=["codigo_ocorrencia1", "codigo_ocorrencia3", "codigo_ocorrencia4", "ocorrencia_latitude",
                 "ocorrencia_longitude", "ocorrencia_pais", "investigacao_aeronave_liberada", "investigacao_status",
                 "divulgacao_relatorio_numero", "divulgacao_relatorio_publicado", "divulgacao_dia_publicacao",
                 "total_aeronaves_envolvidas", "ocorrencia_saida_pista"], inplace=True)

In [6]:
#Criando esquema para validação dos dados
#validando ocorrencia_hora com expressão regular

schema = pa.DataFrameSchema(
    columns = {
        "codigo":pa.Column(pa.Int, required=False),
        "codigo_ocorrencia":pa.Column(pa.Int, required=False),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String),
        "ocorrencia_aerodromo":pa.Column(pa.String, nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [ ]:
#Validando schema

schema.validate(df)

In [ ]:
#Obter ocorrências com mais de 10 recomendações

filtro = df.total_recomendacoes > 10
df.loc[filtro]

In [ ]:
#Obter ocorrências com mais de 10 recomendações por cidade

filtro = df.total_recomendacoes > 10
df.loc[filtro, "ocorrencia_cidade"]

In [ ]:
#Obter total de ocorrências com mais de 10 recomendações por cidade

filtro = df.total_recomendacoes > 10
df.loc[filtro, ["ocorrencia_cidade", "total_recomendacoes"]]

In [ ]:
#Filtrando ocorrências do tipo "incidente grave"

filtro = df.ocorrencia_classificacao == "INCIDENTE GRAVE"
df.loc[filtro]

In [ ]:
#Filtrando ocorrências do tipo "incidente grave" no estado de SP

filtro = df.ocorrencia_classificacao == "INCIDENTE GRAVE"
filtro1 = df.ocorrencia_uf == "SP"
df.loc[filtro & filtro1]

In [ ]:
#Filtrando ocorrências do tipo "incidente grave" ou "incidente" no estado de SP]

filtro = (df.ocorrencia_classificacao == "INCIDENTE GRAVE") | (df.ocorrencia_classificacao == "INCIDENTE")
#Alternativa: filtro = df.ocorrencia_classificacao.isin(["INCIDENTE GRAVE", "INCIDENTE"])
filtro1 = df.ocorrencia_uf == "SP"
df.loc[filtro & filtro1]

In [ ]:
#Filtrando ocorrências cuja cidade começa com a letra "C"

filtro = df.ocorrencia_cidade.str[0] == "C"
df.loc[filtro]

In [ ]:
#Filtrando ocorrências cuja cidade contém a sílaba "MA"

filtro = df.ocorrencia_cidade.str.contains("MA")
df.loc[filtro]

In [ ]:
#Obter ocorrências apenas de 2015

filtro = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro]

In [ ]:
#Obter ocorrências apenas do mês 12 de 2015

filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 12)
df.loc[filtro]

In [18]:
#Criar coluna com dados das colunas do dia e hora juntas

df["ocorrencia_dia_hora"] = pd.to_datetime(df.ocorrencia_dia.astype(str) + " " + df.ocorrencia_hora)

In [ ]:
#Atribuir ocorrências 2015 mês 03 a um novo df

filtro = df.ocorrencia_dia.dt.year == 2015
filtro1 = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro & filtro1]
df201503

In [ ]:
#verificando valores nulos no novo df

df201503.count()

In [ ]:
#Agrupamento por código da ocorrencia

df201503.groupby(["codigo_ocorrencia"]).codigo_ocorrencia.count()

In [ ]:
#Contar classificação da ocorrência mediante o código da ocorrência

df201503.groupby(["ocorrencia_classificacao"]).codigo_ocorrencia.count()

In [ ]:
#Contar classificação da ocorrência mediante o código do aerodromo

df201503.groupby(["ocorrencia_classificacao"]).ocorrencia_aerodromo.count()     #PS: contar com um coluna que pode ter valor nulo não é uma boa prática

In [ ]:
#Contar classificação da ocorrência mediante registros, e não pelo valor de uma coluna

df201503.groupby(["ocorrencia_classificacao"]).size()       #PS: boa prática

In [ ]:
#Agrupar classificação da ocorrência mediante ordem crescente

df201503.groupby(["ocorrencia_classificacao"]).size().sort_values()     #Ordem descrescente: ".sort_values(ascending=False)"

In [ ]:
#Criando df com ocorrências do ano de 2010 na região Sudeste

filtro = df.ocorrencia_dia.dt.year == 2010
filtro1 = df.ocorrencia_uf.isin(["SP", "RJ", "MG", "ES"])
dfsudeste2010 = df.loc[filtro & filtro1]
dfsudeste2010

In [ ]:
#Obtendo quantidade de cada tipo de ocorrência na região Sudeste no ano de 2010

dfsudeste2010.groupby(["ocorrencia_classificacao"]).size()

In [ ]:
#Obtendo quantidade de cada tipo de ocorrência por estado da região Sudeste no ano de 2010 (output: por tipo de ocorrência)

dfsudeste2010.groupby(["ocorrencia_classificacao", "ocorrencia_uf"]).size()

In [ ]:
#Obtendo quantidade de cada tipo de ocorrência por estado da região Sudeste no ano de 2010 (output: por estado)

dfsudeste2010.groupby(["ocorrencia_uf", "ocorrencia_classificacao"]).size()

In [ ]:
#Obtendo quantidade de cada tipo de ocorrência por cidades dos estados da região Sudeste no ano de 2010

dfsudeste2010.groupby(["ocorrencia_cidade"]).size().sort_values(ascending=False)

In [ ]:
#Obtendo ocorrências para a cidade do Rio 

filtro = dfsudeste2010.ocorrencia_cidade == "RIO DE JANEIRO"
dfsudeste2010.loc[filtro]

In [ ]:
#Obtendo o total de recomendações na cidade do Rio

filtro = dfsudeste2010.ocorrencia_cidade == "RIO DE JANEIRO"
dfsudeste2010.loc[filtro].total_recomendacoes.sum()

#Para verificar se o número total bate com o informado, basta criar um novo filtro
  #filtro = dfsudeste2010.ocorrencia_cidade == "RIO DE JANEIRO"
  #filtro1 = dfsudeste2010.total_recomendacoes > 0
  #dfsudeste2010.loc[filtro & filtro1]

In [ ]:
#Obtendo total de recomendações por cidade da região Sudeste

dfsudeste2010.groupby(["ocorrencia_cidade"]).total_recomendacoes.sum()

#PS: nas colunas com valores não informados, deve ser utilizado "dropna=False" para não haver conflito de valores
  #dfsudeste2010.groupby(["ocorrencia_aerodromo"], dropna=False).total_recomendacoes.sum()

In [44]:
#Obter cidades da região Sudeste que possuem uma ou mais recomendações

filtro = dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro].groupby(["ocorrencia_cidade"]).total_recomendacoes.sum()

ocorrencia_cidade
BELO HORIZONTE            2
BOITUVA                   1
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
CAMANDUCAIA               9
IPERÓ                     3
ITANHAÉM                  1
JUNDIAÍ                   7
NOVO HORIZONTE            3
RESENDE                   8
RIO DE JANEIRO           25
SÃO BERNARDO DO CAMPO     2
SÃO PAULO                41
TIMÓTEO                   3
Name: total_recomendacoes, dtype: int64

In [ ]:
#Agrupar nas cidades o total de recomendações por mês

dfsudeste2010.loc[filtro].groupby(["ocorrencia_cidade", dfsudeste2010.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

#Verificar se número retornado está correto
  #filtro = dfsudeste2010.total_recomendacoes > 0
  #filtro1 = dfsudeste2010.ocorrencia_cidade == "SÃO PAULO"
  #dfsudeste2010.loc[filtro & filtro1]